In [1]:
import polars as pl
pl.Config.set_tbl_rows(50)
pl.Config.set_fmt_str_lengths(100)
df = pl.read_parquet('CurProcessorExport-00001.snappy.parquet') ## data from Nov 11th for this account

## EC2 Products

In [16]:
ec2_non_null_df = df.filter((pl.col('line_item_resource_id').is_not_null())
                            & (pl.col('line_item_product_code').eq('AmazonEC2'))
                            & (pl.col('product_region_code').is_not_null()))
triplets_df = ec2_non_null_df.select(["line_item_product_code", "product_region_code", "line_item_resource_id"])
triplets_df = triplets_df.unique()
triplets_df.sort(["product_region_code", "line_item_resource_id"])
triplets_df

# EBS snapshots ids follow arns with :snapshot/ - Snapshot ID
# EBS volumes start with vol- - Volume ID
# EC2 instances start with i- - Instance ID
# nat gateways ids follow arns with :natgateway/ - Nat Gateway ID
# All else will be marked as Unknown
resource_id_df = triplets_df.with_columns(
    resource_id_type = pl.when(pl.col('line_item_resource_id').str.contains(':snapshot/'))
    .then(pl.lit('Snapshot ID'))
    .when(pl.col('line_item_resource_id').str.starts_with('vol-'))
    .then(pl.lit('Volume ID'))
    .when(pl.col('line_item_resource_id').str.starts_with('i-'))
    .then(pl.lit('Instance ID'))
    .when(pl.col('line_item_resource_id').str.contains(':natgateway/'))
    .then(pl.lit('Nat Gateway ID'))
    .otherwise(pl.lit('Unknown'))
)
# use regex to extract the id from the arn
resource_id_df = resource_id_df.with_columns(
    pl.col("line_item_resource_id").str.replace_all(r'.*:snapshot/', "")
)
resource_id_df = resource_id_df.with_columns(
    pl.col("line_item_resource_id").str.replace_all(r'.*:natgateway/', "")
)
resource_id_df

line_item_product_code,product_region_code,line_item_resource_id,resource_id_type
str,str,str,str
"""AmazonEC2""","""us-east-1""","""snap-0d4bbd677ad8a6d04""","""Snapshot ID"""
"""AmazonEC2""","""us-west-2""","""snap-06fd240a6c81a013d""","""Snapshot ID"""
"""AmazonEC2""","""us-east-1""","""snap-0fc7285cecd8986c8""","""Snapshot ID"""
"""AmazonEC2""","""us-east-1""","""snap-09c4ff6ca7f20c32a""","""Snapshot ID"""
"""AmazonEC2""","""us-east-1""","""vol-0833f255708f2c024""","""Volume ID"""
"""AmazonEC2""","""us-east-1""","""vol-0b5159a3cb42f7861""","""Volume ID"""
"""AmazonEC2""","""us-east-1""","""snap-04d6deecccae4eec8""","""Snapshot ID"""
"""AmazonEC2""","""us-east-1""","""snap-0012b2b72e3de1580""","""Snapshot ID"""
"""AmazonEC2""","""us-east-1""","""snap-00cdd334fa22503c6""","""Snapshot ID"""


## S3 Products

In [17]:
s3_non_null_df = df.filter((pl.col('line_item_resource_id').is_not_null())
                            & (pl.col('line_item_product_code').eq('AmazonS3'))
                            & (pl.col('product_region_code').is_not_null()))
triplets_df = s3_non_null_df.select(["line_item_product_code", "product_region_code", "line_item_resource_id"])
triplets_df = triplets_df.unique()
triplets_df.sort(["product_region_code", "line_item_resource_id"])
triplets_df

# S3 bucket names are the resource id
resource_id_df = triplets_df.with_columns(
    resource_id_type = pl.lit('Bucket Name')
)
resource_id_df

line_item_product_code,product_region_code,line_item_resource_id,resource_id_type
str,str,str,str
"""AmazonS3""","""ca-central-1""","""poc-infinitt-dotnet-image-builder-logs""","""Bucket Name"""
"""AmazonS3""","""us-east-1""","""winson-test-version""","""Bucket Name"""
"""AmazonS3""","""us-east-1""","""bigquerys3nr""","""Bucket Name"""
"""AmazonS3""","""us-west-2""","""aws-focus-187940856853""","""Bucket Name"""
"""AmazonS3""","""us-east-1""","""ae-json-test-20231130""","""Bucket Name"""
"""AmazonS3""","""us-east-1""","""sql-server-backup-bucket-mk""","""Bucket Name"""
"""AmazonS3""","""us-east-2""","""2022-intern-rlewis""","""Bucket Name"""
"""AmazonS3""","""us-east-2""","""serverless-deplopyment-idfc""","""Bucket Name"""
"""AmazonS3""","""us-east-2""","""cw-syn-results-187940856853-us-east-2""","""Bucket Name"""


##

## Lambda Products

In [18]:
## get triplets of product code, region code, and resource id for Usage and SavingsPlanCoveredUsage
lam_df = df.filter((pl.col('line_item_product_code').eq('AWSLambda')) &
                (pl.col('product_region_code').is_not_null()) &
                (pl.col('line_item_resource_id').is_not_null())
)
triplets_df = lam_df.select(["line_item_product_code", "product_region_code", "line_item_resource_id"])
triplets_df = triplets_df.unique()
triplets_df.sort(["product_region_code", "line_item_resource_id"])

# Lambda tags are extracted based on the full arn in the resource id
resource_id_df = triplets_df.with_columns(
    resource_id_type = pl.lit('Lambda ARN')
)
resource_id_df

line_item_product_code,product_region_code,line_item_resource_id,resource_id_type
str,str,str,str
"""AWSLambda""","""us-east-1""","""arn:aws:lambda:us-east-1:187940856853:function:CurProcessorStack-CURProcessingLambda1529E409-w4JkLjl…","""Lambda ARN"""
"""AWSLambda""","""us-east-1""","""arn:aws:lambda:us-east-1:187940856853:function:cwsyn-hardees-reward-east-1-e609bd86-a68a-4ab0-a54e-4…","""Lambda ARN"""
"""AWSLambda""","""us-west-2""","""arn:aws:lambda:us-west-2:187940856853:function:aaguiar-ad-reader-pwsh""","""Lambda ARN"""
"""AWSLambda""","""us-east-1""","""arn:aws:lambda:us-east-1:187940856853:function:2ndwatchtestuserreport""","""Lambda ARN"""
"""AWSLambda""","""us-east-1""","""arn:aws:lambda:us-east-1:187940856853:function:cwsyn-hardees-join-east-1-c935bed2-a0a7-4eea-86a2-b00…","""Lambda ARN"""
"""AWSLambda""","""us-east-1""","""arn:aws:lambda:us-east-1:187940856853:function:identify-differences-fn""","""Lambda ARN"""
"""AWSLambda""","""us-west-2""","""arn:aws:lambda:us-west-2:187940856853:function:aaguiar-ad-reader""","""Lambda ARN"""
"""AWSLambda""","""us-east-1""","""arn:aws:lambda:us-east-1:187940856853:function:2w-nr-disable-muting-rule-function""","""Lambda ARN"""
"""AWSLambda""","""us-west-1""","""arn:aws:lambda:us-west-1:187940856853:function:cwsyn-carlsjr-menu-west-1-c40ab3ed-ebc6-41c7-80c4-c1f…","""Lambda ARN"""


## RDS Products

In [19]:
## get triplets of product code, region code, and resource id for Usage and SavingsPlanCoveredUsage
rds_df = df.filter((pl.col('line_item_product_code').eq('AmazonRDS')) &
                (pl.col('product_region_code').is_not_null()) &
                (pl.col('line_item_resource_id').is_not_null())
)
# usage_df = rds_df.filter(
#     (pl.col('line_item_line_item_type').is_in(['Usage', 'DiscountedUsage', 'SavingPlanCoveredUsage'])) &
#     (~pl.col('line_item_usage_type').str.contains('Bytes'))
# )
triplets_df = rds_df.select(["line_item_product_code",
                            "product_region_code",
                            "line_item_resource_id"
                            ])
triplets_df = triplets_df.unique()
triplets_df.sort(["product_region_code", "line_item_resource_id"])
triplets_df

resource_id_df = triplets_df.with_columns(
    resource_id_type = pl.when(pl.col('line_item_resource_id').str.contains(':cluster-snapshot:'))
    .then(pl.lit('Snapshot ARN'))
    .when(pl.col('line_item_resource_id').str.contains(':db:'))
    .then(pl.lit('Database ARN'))
    .when(pl.col('line_item_resource_id').str.contains(':cluster:'))
    .then(pl.lit('Cluster ARN'))
    .otherwise(pl.lit('Unknown'))
)
resource_id_df

line_item_product_code,product_region_code,line_item_resource_id,resource_id_type
str,str,str,str
"""AmazonRDS""","""eu-north-1""","""arn:aws:rds:eu-north-1:187940856853:cluster:cluster-6tcvzlm3edthuwrnqvtmxrq5na""","""Cluster ARN"""
"""AmazonRDS""","""eu-north-1""","""arn:aws:rds:eu-north-1:187940856853:db:database-1-instance-1""","""Database ARN"""
"""AmazonRDS""","""us-east-2""","""arn:aws:rds:us-east-2:187940856853:db:dn-wordpress-database""","""Database ARN"""
"""AmazonRDS""","""us-west-2""","""arn:aws:rds:us-west-2:187940856853:db:dn-wordpress-database-replica""","""Database ARN"""
"""AmazonRDS""","""us-east-1""","""arn:aws:rds:us-east-1:187940856853:db:database-1""","""Database ARN"""
"""AmazonRDS""","""us-east-2""","""arn:aws:rds:us-east-2:187940856853:cluster:cluster-t3hrwc4gf3zkly36x5diw5u6di""","""Cluster ARN"""
"""AmazonRDS""","""us-east-1""","""arn:aws:rds:us-east-1:187940856853:db:sql-rds-enterprise-not-encrypted""","""Database ARN"""
"""AmazonRDS""","""eu-north-1""","""arn:aws:rds:eu-north-1:187940856853:cluster-snapshot:database-1-final-snapshot""","""Snapshot ARN"""
"""AmazonRDS""","""us-east-1""","""arn:aws:rds:us-east-1:187940856853:db:db-sql-rds-express""","""Database ARN"""


## Redshift Products

In [13]:
red_df = df.filter((pl.col('line_item_product_code').eq('AmazonRedshift')) &
                (pl.col('product_region_code').is_not_null()) &
                (pl.col('line_item_resource_id').is_not_null())
)
triplets_df = red_df.select(["line_item_product_code", "product_region_code", "line_item_resource_id"])
triplets_df = triplets_df.unique()
triplets_df.sort(["product_region_code", "line_item_resource_id"])
triplets_df

resource_id_df = triplets_df.with_columns(
    resource_id_type = pl.when(pl.col('line_item_resource_id').str.contains(':cluster:'))
    .then(pl.lit('Cluster ARN'))
    .when(pl.col('line_item_resource_id').str.contains(':redshift-serverless:'))
    .then(pl.lit('Serverless ARN'))
    .otherwise(pl.lit('Unknown'))
)
resource_id_df

line_item_product_code,product_region_code,line_item_resource_id,resource_id_type
str,str,str,str
"""AmazonRedshift""","""us-east-1""","""arn:aws:redshift:us-east-1:999537480004:cluster:source-to-65127672""","""Cluster ARN"""
"""AmazonRedshift""","""us-west-2""","""arn:aws:redshift:us-west-2:187940856853:cluster:redshift-cluster-1""","""Cluster ARN"""
"""AmazonRedshift""","""us-east-1""","""arn:aws:redshift-serverless:us-east-1:187940856853:namespace/7005ab5e-d1da-457f-a1d8-508e04c36eb3""","""Serverless ARN"""
"""AmazonRedshift""","""us-east-1""","""arn:aws:redshift:us-east-1:187940856853:cluster:redshift-cluster-1""","""Cluster ARN"""


## ECR Products

In [15]:
ecr_df = df.filter((pl.col('line_item_product_code').eq('AmazonECR')) &
                (pl.col('product_region_code').is_not_null()) &
                (pl.col('line_item_resource_id').is_not_null())
)
triplets_df = ecr_df.select(["line_item_product_code", "product_region_code", "line_item_resource_id"])
triplets_df = triplets_df.unique()
triplets_df.sort(["product_region_code", "line_item_resource_id"])
triplets_df

# resource_id_df = triplets_df.with_columns(
#     resource_id_type = pl.when(pl.col('line_item_resource_id').str.contains(':cluster:'))
#     .then(pl.lit('Cluster ARN'))
#     .when(pl.col('line_item_resource_id').str.contains(':redshift-serverless:'))
#     .then(pl.lit('Serverless ARN'))
#     .otherwise(pl.lit('Unknown'))
# )
# resource_id_df

line_item_product_code,product_region_code,line_item_resource_id
str,str,str
"""AmazonECR""","""us-east-1""","""arn:aws:ecr:us-east-1:187940856853:repository/cdk-hnb659fds-container-assets-187940856853-us-east-1"""
"""AmazonECR""","""us-east-1""","""arn:aws:ecr:us-east-1:187940856853:repository/ccrepl2"""
"""AmazonECR""","""ap-southeast-1""","""arn:aws:ecr:ap-southeast-1:187940856853:repository/rey-ecr"""
"""AmazonECR""","""us-east-1""","""arn:aws:ecr:us-east-1:187940856853:repository/ccrepl"""
"""AmazonECR""","""us-west-2""","""arn:aws:ecr:us-west-2:187940856853:repository/focus_convert"""
"""AmazonECR""","""us-west-2""","""arn:aws:ecr:us-west-2:187940856853:repository/focus_billing_pipeline_etl"""
"""AmazonECR""","""ca-central-1""","""arn:aws:ecr:ca-central-1:187940856853:repository/ssmrepair841fc95a/ssmrepairfunctiona754f442repo"""
"""AmazonECR""","""us-east-1""","""arn:aws:ecr:us-east-1:187940856853:repository/say-something"""
